## Pubannotation Json Example
## April 27

In [20]:
import pandas as pd

df = pd.read_csv ('corona/manuscript/Supplemental_file6.csv')
print (df)

         pmid        type   id       entity_type  \
0    31991541  annotation   A2       Virus_other   
1    31991541  annotation   A3  Virus_SARS-CoV-2   
2    31991541  annotation   A4  Virus_SARS-CoV-2   
3    31991541  annotation   A5      Virus_family   
4    31991541  annotation   A1  Virus_SARS-CoV-2   
..        ...         ...  ...               ...   
150  31996494  annotation  A60  Virus_SARS-CoV-2   
151  31996494  annotation  A61      Virus_family   
152  31996494  annotation  A62      Virus_family   
153  31996494  annotation  A63  Virus_SARS-CoV-2   
154  31996494  annotation  A64  Virus_SARS-CoV-2   

                                          text location/reference  \
0                                     SARS-CoV              114:8   
1    most recent emergent group 2B coronavirus             293:41   
2                                    2019-nCoV               27:9   
3                                  Coronavirus              14:11   
4                            n

In [193]:

import collections
import csv
import sys
import codecs
from xml.dom.minidom import parse
import xml.dom.minidom
from bs4 import BeautifulSoup


with codecs.open("corona/manuscript/Supplemental_file4.xml", "r", "utf-8") as file:##gold standard
    soup = BeautifulSoup(file, "html.parser")
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE collection SYSTEM "BioC.dtd">
<collection>
 <source/>
 Pubmed
 <date>
  20200306
 </date>
 <key>
 </key>
 <infon key="BQ_URL">
  http://www.bioqrator.org
 </infon>
 <infon key="BQ_NAME">
  Corona_gold
 </infon>
 <document>
  <id>
   31991541
  </id>
  <infon key="BQ_FROM">
   PubMed
  </infon>
  <infon key="BQ_PIESCORE">
   -0.4251
  </infon>
  <infon key="BQ_DONE">
   NO
  </infon>
  <infon key="BQ_CURATABLE">
   YES
  </infon>
  <passage>
   <infon key="type">
    title
   </infon>
   <offset>
    0
   </offset>
   <text>
    Return of the Coronavirus: 2019-nCoV.
   </text>
   <annotation id="A4">
    <infon key="representative_name">
     SARS-CoV-2
    </infon>
    <infon key="type">
     Virus_SARS-CoV-2
    </infon>
    <infon key="NCBI:txid">
     2697049
    </infon>
    <location length="9" offset="27">
    </location>
    <text>
     2019-nCoV
    </text>
   </annotation>
   <annotation id="A5">
    <infon key="representative_n

In [225]:
from xml.etree import ElementTree

with open('corona/manuscript/Supplemental_file4.xml', 'rt') as f:
    tree = ElementTree.parse(f)

source_id = []
source_db = []
text = []
denotation_array = []
annotation_id = []
begin = []
end = []
for node in tree.iter():
    #print(node.tag, node.attrib, node.text)
    if node.tag == "id":
        source_id.append(node.text)
        source_id.append(node.text)
    if node.tag =="infon" and node.attrib['key']=='BQ_FROM':
        source_db.append(node.text)
        source_db.append(node.text)
    if node.tag == 'annotation':
        annotation_id.append(node.attrib.get('id'))
    if node.tag=='location':
        span_begin = int(node.attrib.get('offset'))
        span_end   = int(span_begin) + int(node.attrib.get('length'))
        begin.append(span_begin)
        end.append(span_end)

###############################################################################
#print(text)

print('done!')                                                         
with open('Gold_std_Pubannotation_2.json', 'w') as out_file: 
    ann_id_ctr = 0 
    general_ctr = 0
    documents = soup.find_all('document')
    #print(documents[1])
    for i in documents:
        passages = i.find_all('passage')
        for k in passages:
            offset = int(k.find('offset').text)
            passage_text = k.find('text').text
            objects = k.find_all('annotation')
            denotation_array = []
            for obj in objects:
                obj_id = annotation_id[ann_id_ctr]
                info = obj.find_all('infon')
                for ann_info in info:
                    if "key=\"type\"" in str(ann_info):
                        obj_description = str(ann_info.text)+':'
                    if "NCBI:txid" in str(ann_info):
                        obj_description = obj_description + str(ann_info.text)
                span_begin = begin[ann_id_ctr]-offset
                span_end   = end[ann_id_ctr]-offset
                ann_id_ctr = ann_id_ctr+1
                denotation_array.append({"id": obj_id, "span": {"begin": span_begin, "end": span_end}, "obj": obj_description})                                
         
            tmp_dict = {
                   
                    "text": passage_text, # either abstract or title
                    "sourcedb": source_db[general_ctr],
                    "sourceid": source_id[general_ctr],
                    "denotations": denotation_array  # all annotations
                }
            
            general_ctr = general_ctr + 1
            
         
    #print(json.dumps(tmp_dict,indent=4),file=out_file,sep='\n')
            out_file.write(json.dumps(tmp_dict))
            out_file.write('\n')
                                         
        


done!


In [224]:

with codecs.open("Gold_std_Pubannotation_2.json", "r", "utf-8") as file:
    soupjson = BeautifulSoup(file, "html.parser")
print(soupjson.prettify())

{"text": "Return of the Coronavirus: 2019-nCoV. ", "sourcedb": "PubMed", "sourceid": "31991541", "denotations": [{"id": "A4", "span": {"begin": 27, "end": 36}, "obj": "Virus_SARS-CoV-2:2697049"}, {"id": "A5", "span": {"begin": 14, "end": 25}, "obj": "Virus_family:693995"}]}
{"text": "The emergence of a novel coronavirus (2019-nCoV) has awakened the echoes of SARS-CoV from nearly two decades ago. Yet, with technological advances and important lessons gained from previous outbreaks, perhaps the world is better equipped to deal with the most recent emergent group 2B coronavirus. ", "sourcedb": "PubMed", "sourceid": "31991541", "denotations": [{"id": "A2", "span": {"begin": 76, "end": 84}, "obj": "Virus_other:694009 "}, {"id": "A3", "span": {"begin": 255, "end": 296}, "obj": "Virus_SARS-CoV-2:2697049"}, {"id": "A1", "span": {"begin": 19, "end": 36}, "obj": "Virus_SARS-CoV-2:2697049"}, {"id": "A6", "span": {"begin": 38, "end": 47}, "obj": "Virus_SARS-CoV-2:2697049"}]}
{"text": "Mast cells c

In [209]:
import json

JsonList = []
print("Started Reading JSON file which contains multiple JSON document")
with open('Pub.json') as json_file:
       for j in json_file:
            x = json.loads(j)
            JsonList.append(x)

print(JsonList[19])

Started Reading JSON file which contains multiple JSON document
{'text': 'As of January 22, 2020, a total of 571 cases of the 2019-new coronavirus (2019-nCoV) have been reported in 25 provinces (districts and cities) in China. At present, there is no vaccine or antiviral treatment for human and animal coronavirus, so that identifying the drug treatment options as soon as possible is critical for the response to the 2019-nCoV outbreak. Three general methods, which include existing broad-spectrum antiviral drugs using standard assays, screening of a chemical library containing many existing compounds or databases, and the redevelopment of new specific drugs based on the genome and biophysical understanding of individual coronaviruses, are used to discover the potential antiviral treatment of human pathogen coronavirus. Lopinavir /Ritonavir, Nucleoside analogues, Neuraminidase inhibitors, Remdesivir, peptide (EK1), arbidol, RNA synthesis inhibitors (such as TDF, 3TC), anti-inflammatory dr